In [6]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from collections import Counter
import joblib

# Directory for the dataset
data_dir = "/Users/samikazi/Desktop/GithubProject/dataset"

# Set image parameters
img_height, img_width = 128, 128

# Helper function to load and preprocess images
def load_images_from_directory(directory, target_size):
    images = []
    labels = []
    class_names = sorted(os.listdir(directory))
    
    for class_index, class_name in enumerate(class_names):
        class_path = os.path.join(directory, class_name)
        
        if os.path.isdir(class_path):
            class_images = []
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                try:
                    img = load_img(img_path, target_size=target_size)
                    img_array = img_to_array(img)
                    img_array = img_array.flatten()  # Flatten the 2D image to a 1D array
                    class_images.append(img_array)
                except Exception as e:
                    print(f"Could not load image {img_path}: {e}")
            
            # Add class images if it has at least two samples
            if len(class_images) >= 2:
                images.extend(class_images)
                labels.extend([class_index] * len(class_images))
            else:
                print(f"Skipping class '{class_name}' due to insufficient images.")
                    
    return np.array(images), np.array(labels), class_names

# Load and preprocess images
images, labels, class_names = load_images_from_directory(data_dir, target_size=(img_height, img_width))

# Check class distribution after filtering
class_counts = Counter(labels)
print("Class distribution after filtering:", class_counts)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels
)

# Initialize and train a Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions and calculate accuracy on the validation set
y_pred = rf_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

# Save the trained model
joblib.dump(rf_model, "animal_classifier_rf_model.pkl")
print("Model saved as 'animal_classifier_rf_model.pkl'")


Skipping class 'cat' due to insufficient images.
Class distribution after filtering: Counter({2: 181, 5: 176, 4: 169, 3: 3})
Validation Accuracy: 88.68%
Model saved as 'animal_classifier_rf_model.pkl'


In [5]:
import os
from icrawler.builtin import GoogleImageCrawler
import threading
import time

# Create dataset folders
def create_folders(main_folder, classes):
    os.makedirs(main_folder, exist_ok=True)
    for class_name in classes:
        os.makedirs(os.path.join(main_folder, class_name), exist_ok=True)
    print(f"Folders created for classes: {', '.join(classes)}")

# Download images for a single class with retry logic
def download_images_for_class(class_name, limit, main_folder):
    output_dir = os.path.join(main_folder, class_name)
    max_attempts = 5
    downloaded = 0

    for attempt in range(max_attempts):
        try:
            print(f"[{class_name}] Attempt {attempt + 1}: Downloading images...")
            crawler = GoogleImageCrawler(
                feeder_threads=1,
                parser_threads=2,
                downloader_threads=4,
                storage={'root_dir': output_dir}
            )
            crawler.crawl(
                keyword=class_name,
                max_num=limit - downloaded,
                min_size=(200, 200),
                max_size=None,
                file_idx_offset='auto'
            )
            
            downloaded = len([f for f in os.listdir(output_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
            print(f"[{class_name}] {downloaded}/{limit} images downloaded.")

            if downloaded >= limit:
                break

        except Exception as e:
            print(f"[{class_name}] Error during attempt {attempt + 1}: {e}")
            time.sleep(2)  # Wait before retrying

    if downloaded < limit:
        print(f"[{class_name}] Warning: Only {downloaded}/{limit} images downloaded.")

# Main function to manage the download process
def main():
    # Define classes and dataset folder
    main_folder = "./dataset"
    classes = ['dog', 'cow', 'cat', 'lamb', 'zebra']
    images_per_class = 100

    # Create folders for dataset
    create_folders(main_folder, classes)

    # Use threads to download images concurrently
    threads = []
    for class_name in classes:
        thread = threading.Thread(target=download_images_for_class, args=(class_name, images_per_class, main_folder))
        threads.append(thread)
        thread.start()

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    print("Image downloading completed!")

if __name__ == "__main__":
    main()


2024-11-25 16:30:11,247 - INFO - icrawler.crawler - start crawling...
2024-11-25 16:30:11,249 - INFO - icrawler.crawler - start crawling...
2024-11-25 16:30:11,250 - INFO - icrawler.crawler - starting 1 feeder threads...
2024-11-25 16:30:11,250 - INFO - icrawler.crawler - start crawling...
2024-11-25 16:30:11,252 - INFO - icrawler.crawler - start crawling...
2024-11-25 16:30:11,253 - INFO - icrawler.crawler - start crawling...
2024-11-25 16:30:11,253 - INFO - icrawler.crawler - starting 1 feeder threads...
2024-11-25 16:30:11,254 - INFO - feeder - thread feeder-001 exit
2024-11-25 16:30:11,254 - INFO - icrawler.crawler - starting 2 parser threads...
2024-11-25 16:30:11,254 - INFO - icrawler.crawler - starting 1 feeder threads...
2024-11-25 16:30:11,255 - INFO - icrawler.crawler - starting 1 feeder threads...
2024-11-25 16:30:11,255 - INFO - icrawler.crawler - starting 1 feeder threads...
2024-11-25 16:30:11,257 - INFO - feeder - thread feeder-001 exit
2024-11-25 16:30:11,257 - INFO - i

Folders created for classes: dog, cow, cat, lamb, zebra
[dog] Attempt 1: Downloading images...
[cow] Attempt 1: Downloading images...
[cat] Attempt 1: Downloading images...
[lamb] Attempt 1: Downloading images...
[zebra] Attempt 1: Downloading images...


2024-11-25 16:30:12,637 - INFO - parser - parsing result page https://www.google.com/search?q=dog&ijn=0&start=0&tbs=&tbm=isch
2024-11-25 16:30:12,792 - INFO - parser - parsing result page https://www.google.com/search?q=zebra&ijn=0&start=0&tbs=&tbm=isch
2024-11-25 16:30:12,918 - INFO - parser - parsing result page https://www.google.com/search?q=cow&ijn=0&start=0&tbs=&tbm=isch
2024-11-25 16:30:13,031 - INFO - downloader - image #1	https://cdn.mos.cms.futurecdn.net/HjFE8NKWuCmgfHCcndJ3rK-1200-80.jpg
2024-11-25 16:30:13,114 - INFO - downloader - image #1	https://cdn.britannica.com/23/523-050-0C120420/cow-Holstein-Friesian.jpg
2024-11-25 16:30:13,148 - INFO - parser - parsing result page https://www.google.com/search?q=lamb&ijn=0&start=0&tbs=&tbm=isch
2024-11-25 16:30:13,207 - INFO - downloader - image #2	https://cdn.britannica.com/55/174255-050-526314B6/brown-Guernsey-cow.jpg
2024-11-25 16:30:13,265 - INFO - parser - no more page urls for thread parser-002 to parse
2024-11-25 16:30:13,26

[dog] 1/100 images downloaded.
[dog] Attempt 2: Downloading images...


2024-11-25 16:30:18,755 - INFO - downloader - image #14	https://cdn.britannica.com/53/157153-050-E5582B5A/Holstein-cow.jpg
2024-11-25 16:30:18,986 - INFO - downloader - image #4	https://static.wikia.nocookie.net/my-first-encyclopedia/images/6/60/Equus_quagga.jpg
2024-11-25 16:30:19,105 - INFO - downloader - image #15	https://a-z-animals.com/media/2022/10/Fleckvieh-Cattle-2-1024x683.jpg
2024-11-25 16:30:19,204 - INFO - downloader - image #5	https://i.natgeofe.com/n/14dd7036-78ee-46f4-b620-d5eaf7cebda7/zebra-akagera.jpg
2024-11-25 16:30:19,339 - INFO - downloader - image #7	https://images.getrecipekit.com/v1615995124_RedRubbedBabyLambChopsPg101_xyzuwo.jpg
2024-11-25 16:30:19,527 - INFO - downloader - image #8	https://images.squarespace-cdn.com/content/v1/61c2724e42bafd1109690e02/548b66da-5899-476c-aebf-22c5423eff2e/Lamb+with+tail+-+end+live+lamb+cutting.jpg
2024-11-25 16:30:19,532 - INFO - downloader - no more download task for thread downloader-001
2024-11-25 16:30:19,534 - INFO - downl

...

[lamb] 169/100 images downloaded.


2024-11-25 16:31:02,845 - INFO - downloader - no more download task for thread downloader-003
2024-11-25 16:31:02,846 - INFO - downloader - thread downloader-003 exit
2024-11-25 16:31:02,938 - INFO - downloader - image #54	https://i.redd.it/x8h3agfhipt81.jpg
2024-11-25 16:31:03,166 - INFO - downloader - no more download task for thread downloader-001
2024-11-25 16:31:03,168 - INFO - downloader - thread downloader-001 exit
2024-11-25 16:31:03,611 - INFO - downloader - image #55	https://images.rzss.org.uk/media/Edinburgh_Zoo/EZ_animals/Grevy's_zebra/grevy's%20zebra%204.jpg
2024-11-25 16:31:03,747 - INFO - downloader - image #56	https://static01.nyt.com/images/2024/05/04/multimedia/04xp-zebra-kqfg/04xp-zebra-kqfg-mediumSquareAt3X.jpg
2024-11-25 16:31:04,177 - INFO - downloader - image #57	https://www.awf.org/sites/default/files/styles/species_image/public/2020-04/Website_SpeciesPage_GrevysZebra02_Challenges.jpg
2024-11-25 16:31:04,578 - INFO - downloader - image #58	https://images.twinkl.

[cow] 181/100 images downloaded.


2024-11-25 16:31:06,773 - INFO - downloader - image #65	https://media.greenmatters.com/brand-img/dGyFOJJmN/0x0/international-zebra-day6-1611606399511.jpg
2024-11-25 16:31:07,180 - INFO - downloader - image #66	https://www.krugerpark.co.za/images/zebra-herd-roger-delaharpe-590x390.jpg
2024-11-25 16:31:07,236 - INFO - downloader - image #67	https://img.freepik.com/free-vector/cute-zebra-cartoon-animal-character_1308-154268.jpg
2024-11-25 16:31:07,385 - INFO - downloader - image #68	https://wildtimes.buildingourzoo.com/wp-content/uploads/2022/01/Copy-of-Copy-of-Zebra-mom-_-baby.jpg
2024-11-25 16:31:08,976 - INFO - downloader - image #69	https://cdn.britannica.com/85/219885-050-92BF3722/two-zebras-Serengeti-National-Park-Tanzania.jpg
2024-11-25 16:31:09,932 - ERROR - downloader - Exception caught when downloading file https://www.kansascity.com/latest-news/brahna/picture291929150/alternates/LANDSCAPE_1140/Screenshot%202024-09-04%20115435.png, error: HTTPSConnectionPool(host='www.kansascity

[zebra] 176/100 images downloaded.
Image downloading completed!
